In [1]:
from collections.abc import Iterable

import pymongo
from collections import Counter

In [2]:
c = pymongo.MongoClient()

In [3]:
db0 = c.neuro1337
db1 = c.neuro1338
db2 = c.neuro1339

In [4]:
class CoolDict(dict):
    def __getattr__(self, name):
        attr = self[name]
        if type(attr) is dict:
            return CoolDict(attr)
        elif type(attr) is list:
            return CoolList(attr)
        return attr
    
    
class CoolList(list):
    def __getitem__(self, i):
        item = super().__getitem__(i)
        if type(item) is dict:
            return CoolDict(item)
        elif type(item) is list:
            return CoolList(item)
        return item

    def __getattr__(self, name):
        return CoolList(getattr(x, name) for x in self)


class Block(CoolDict):
    def __init__(self, arg, db=db0):
        self.db = db
        if type(arg) == dict:
            super().__init__(arg)
        elif type(arg) == str:
            super().__init__(db.blocks.find_one({"block.header.id.data": arg}))
        elif type(arg) == int:
            super().__init__(db.blocks.find_one({"block.header.height": arg}))

    @property
    def previousAssembly(self):
        return Assembly(self.previousAssemblyId.data, db=self.db)
    
    @property
    def previousBlock(self):
        return Block(self.block.header.previousBlockHash.data, db=self.db)

    @property
    def transactions(self):
        result = []
        for t in self.db.transactions.find(
            {"blockId.data": self.block.header.id.data, "isCoinbase": False}
        ):
            result.append(Transaction(t))
        return result

    @property
    def coinbase(self):
        return Transaction(
            self.db.transactions.find_one(
                {"blockId.data": self.block.header.id.data, "isCoinbase": True}
            )
        )


class Assembly(CoolDict):
    def __init__(self, arg, db=db0):
        self.db = db
        if type(arg) == str:
            super().__init__(db.assemblies.find_one({"id.data": arg}))
        elif type(arg) == int:
            super().__init__(db.assemblies.find_one({"height": arg}))

    @property
    def previousAssembly(self):
        return Assembly(self.previousAssemblyId.data, db=self.db)


class Transaction(CoolDict):
    def __init__(self, arg, db=db0, pool=True):
        self.db = db
        if type(arg) == dict:
            super().__init__(arg)
        else:
            query = {"transaction.id.data": arg}
            if not pool:
                query["blockId"] = {"$exists": True}
            super().__init__(db.transactions.find_one(query))

    @property
    def block(self):
        return Block(self.blockId.data, db=self.db)




class Transactions(CoolList):
    def __init__(self, arg=None, db=db0, pool=True):
        if type(arg) is str:
            query = {"transaction.id.data": arg}
            if not pool:
                query["blockId"] = {"$exists": True}
        elif type(arg) is dict:
            query = arg
        else:
            query = {}
        transactions = (Transaction(t) for t in db.transactions.find(query))
        super().__init__(transactions)

    @classmethod
    def by_input(cls, transaction_id, output_id, pool=True):
        query = {
            "transaction.inputs": {
                "$elemMatch": {
                    "id.data": transaction_id,
                    "outputId": output_id,
                }
            }
        }
        if not pool:
            query["blockId"] = {"$exists": True}
        return cls(query)


class Blocks(CoolList):
    def __init__(self, arg=None, db=db0):
        self.db = db0
        if isinstance(arg, Iterable):
            super().__init__(arg)
        else:
            if type(arg) == str:
                query = {"block.header.id.data": arg}
            elif type(arg) == int:
                query = {"block.header.height": arg}
            elif type(arg) == dict:
                query = arg
            else:
                query = {}
            super().__init__([Block(b) for b in db.blocks.find(query)])
    
    @classmethod
    def get_tip(cls, db=db0):
        return Block(db.blocks.find().sort("score", pymongo.DESCENDING).next())

In [5]:
Counter(Blocks().block.header.author.keyPub.rawData)

Counter({'Awqh/GaK+zXAydO8h4sDf4apFNmW8SFaIqRqXioYDXy8': 87,
         'A+BPdJmNMRu7dZ0O4+b/jG5CyuLeI870VKYu0DrtJ8I8': 75,
         'AyMKGnZhazXAoXb0SJIqi4Ho+lk6bw0UttE34PDtDbya': 78})

In [208]:
blocks = Blocks()

In [209]:
blocks[10].transactions

[{'_id': ObjectId('5cdd4d7985699a37350740fe'),
  'transaction': {'id': {'type': 'SHA256',
    'data': '+8YKoBO2WH48YIYy0Zlto0+ng1B2XCWDrww698TkBg0='},
   'inputs': [{'id': {'type': 'SHA256',
      'data': 'YRI/oYLChiXWY5r2TXmEZLbHPvjCONCCGScusb2ki88='},
     'outputId': 1,
     'signatureId': 0}],
   'outputs': [{'address': {'data': 'N2DJgbU3v8tbbzaDdBEyn5fPnLZVzAFC6E'},
     'value': {'value': '20150'}},
    {'address': {'data': 'N3kfoQECGGnwsPp5uDqKJXoxjNRdpw8ML7'},
     'value': {'value': '20150'}}],
   'signatures': [{'signature': {'type': 'SHA256',
      'data': 'aF4EAzq9P2dI0NkG0ymwuwbxVHKnwPYZ+uMSx4xNp1KJkcGx3C38oszbDshpvmBrpGggTW/O9KnjjDOtSK52ng=='},
     'keyPub': {'type': 'ECP256K1',
      'rawData': 'A/onLQq4MiXW88WcElvVwuglgo26edpKwnEGverU8qpL'}}]},
  'isCoinbase': False,
  'blockId': {'type': 'SHA256',
   'data': 'qRmznA1gT236hM8PZq+IuB8xIdNA9ehRP1mvmx+TBYk='}},
 {'_id': ObjectId('5cdd4d7985699a37350740ff'),
  'transaction': {'id': {'type': 'SHA256',
    'data': '3rN91RFl3

In [207]:
[t for t in blocks[10].transactions if t.transaction.signatures[0].signature.data.startswith("aF4")]

[{'_id': ObjectId('5cdd4d7985699a37350740fe'),
  'transaction': {'id': {'type': 'SHA256',
    'data': '+8YKoBO2WH48YIYy0Zlto0+ng1B2XCWDrww698TkBg0='},
   'inputs': [{'id': {'type': 'SHA256',
      'data': 'YRI/oYLChiXWY5r2TXmEZLbHPvjCONCCGScusb2ki88='},
     'outputId': 1,
     'signatureId': 0}],
   'outputs': [{'address': {'data': 'N2DJgbU3v8tbbzaDdBEyn5fPnLZVzAFC6E'},
     'value': {'value': '20150'}},
    {'address': {'data': 'N3kfoQECGGnwsPp5uDqKJXoxjNRdpw8ML7'},
     'value': {'value': '20150'}}],
   'signatures': [{'signature': {'type': 'SHA256',
      'data': 'aF4EAzq9P2dI0NkG0ymwuwbxVHKnwPYZ+uMSx4xNp1KJkcGx3C38oszbDshpvmBrpGggTW/O9KnjjDOtSK52ng=='},
     'keyPub': {'type': 'ECP256K1',
      'rawData': 'A/onLQq4MiXW88WcElvVwuglgo26edpKwnEGverU8qpL'}}]},
  'isCoinbase': False,
  'blockId': {'type': 'SHA256',
   'data': 'qRmznA1gT236hM8PZq+IuB8xIdNA9ehRP1mvmx+TBYk='}}]

In [50]:
[
    (ob.get("branchPath"), ob["block"]["header"]["height"])
    for ob in db2.blocks.find({"branchPath.branchIds": {"$ne": [0]}})
]

[]

In [51]:
[
    (ob.get("branchPath"), ob["block"]["header"]["height"])
    for ob in db0.blocks.find({"branchPath.branchIds": {"$ne": [0]}})
]

[]

In [52]:
heights = [
    [ob["block"]["header"]["height"] for ob in db.blocks.find()]
    for db in (db0, db1, db2)
]

In [53]:
h0 = set(heights[0])
h1 = set(heights[1])
h2 = set(heights[2])

In [56]:
print(h0 - h1, h1 - h0)
print(h0 - h2, h2 - h0)
print(max(h0), len(h0))

set() set()
set() set()
608721 290


In [210]:
Transactions("WXKuOLzlIs8NmdTdf3CVbWFeXLugCl+iV2USK00zGDs=").block.branchPath

[]

In [127]:
Block("6Kp8smlbNZsqhkH7gVgKvE/CnZQhI7XfAkjfLB6501E=")

{'_id': ObjectId('5cdc0c3485699a1b8f49a9fe'),
 'block': {'header': {'id': {'type': 'SHA256',
    'data': '6Kp8smlbNZsqhkH7gVgKvE/CnZQhI7XfAkjfLB6501E='},
   'timestamp': {'data': 1557924916},
   'previousBlockHash': {'type': 'SHA256',
    'data': '8vvhrXnrhobPqyorpo2u6d6bW41JackRcPsRXz3lrOw='},
   'author': {'signature': {'type': 'SHA256',
     'data': 'Vga//8WLii/hx9PwY0nw3eN1abViBLtm2ddtgWajS6iXWSpwEzG3i+08rqfJeEANAnGOHY4rgI/U+vO73bvFNg=='},
    'keyPub': {'type': 'ECP256K1',
     'rawData': 'AyMKGnZhazXAoXb0SJIqi4Ho+lk6bw0UttE34PDtDbya'}},
   'height': 624528}},
 'branch': 'UNVERIFIED',
 'branchPath': {'branchIds': [1, 0], 'blockNumbers': [255, 0]}}

In [38]:
Transactions("0eLNKWSfT51YkpVC6CZRWY9BOX2AUZIZseQxvFssCWs=", db=db1, pool=False).block.branchPath

[{'branchIds': [0], 'blockNumbers': [5]}]

In [147]:
Transactions("Bpgw/tkmuJFEYIqY7EYnfGpRCPa2uh3YAWRQiKplqPY=").block.branchPath

[{'branchIds': [0], 'blockNumbers': [274]}]

In [16]:
Transactions.by_input(transaction_id="Kd+jycUYhKsZ5ghqUzFq2QP2RCblatq0lf/GtcSDdS4=", output_id=1, pool=False)

[{'_id': ObjectId('5ce2967685699a2c081d83c2'),
  'transaction': {'id': {'type': 'SHA256',
    'data': 'TLpkCnH0MC5MaRz0dNkUdu7ZATCVbzFdbu3u8e9LOqg='},
   'inputs': [{'id': {'type': 'SHA256',
      'data': 'Kd+jycUYhKsZ5ghqUzFq2QP2RCblatq0lf/GtcSDdS4='},
     'outputId': 1,
     'signatureId': 0},
    {'id': {'type': 'SHA256',
      'data': 'N5ks2r/uNMhefqJzO9bCtM6oc+iITyDPs9O4PqyC9Co='},
     'outputId': 0,
     'signatureId': 0},
    {'id': {'type': 'SHA256',
      'data': '8C5eKM9vU8Q6r71+xnSAfBmX7DpcPH/r6M3qbIKsX44='},
     'outputId': 0,
     'signatureId': 0}],
   'outputs': [{'address': {'data': 'N2DJgbU3v8tbbzaDdBEyn5fPnLZVzAFC6E'},
     'value': {'value': '582162'}},
    {'address': {'data': 'N3kfoQECGGnwsPp5uDqKJXoxjNRdpw8ML7'},
     'value': {'value': '582163'}}],
   'signatures': [{'signature': {'type': 'SHA256',
      'data': 'PnaG8sJ1mNDl11obhVooUhAYr2B5q3uQ8JvRAKrtOC23Cwq7fFRu1gd2wUinfTw8Y2jpPDaGhVlsjEgJDdaTnQ=='},
     'keyPub': {'type': 'ECP256K1',
      'rawData': 'Ahj

In [7]:
Transactions.by_input("TLpkCnH0MC5MaRz0dNkUdu7ZATCVbzFdbu3u8e9LOqg=", output_id=0)

[{'_id': ObjectId('5ce2966e85699a2c081d83b7'),
  'transaction': {'id': {'type': 'SHA256',
    'data': '84jnlinty7WczW136n9TJzq91YdtDng6AwHUtqhgt7Y='},
   'inputs': [{'id': {'type': 'SHA256',
      'data': 'TLpkCnH0MC5MaRz0dNkUdu7ZATCVbzFdbu3u8e9LOqg='},
     'outputId': 0,
     'signatureId': 0},
    {'id': {'type': 'SHA256',
      'data': 'RI3FHB4DK8RpWza9AucKVXTPlyaVF0lpx7S+e0k6Mbo='},
     'outputId': 1,
     'signatureId': 0}],
   'outputs': [{'address': {'data': 'NHW276q4ZQvaDTDGWLvnrfrnVsUY18Esjm'},
     'value': {'value': '458775'}},
    {'address': {'data': 'N2DJgbU3v8tbbzaDdBEyn5fPnLZVzAFC6E'},
     'value': {'value': '458775'}}],
   'signatures': [{'signature': {'type': 'SHA256',
      'data': '1eTQxiQzp/YPwDdzMdWZUd6PxKNzgfwMUGwzN+RneyLs3W3XpGvxsgO2D5ni1NAL18tSDxu12LYUSFFt2pcYiQ=='},
     'keyPub': {'type': 'ECP256K1',
      'rawData': 'A7AOi95K1rfyo5tqawwc1chVVl3IXAu2ZxL3MYLGmWQ6'}},
    {'signature': {'type': 'SHA256',
      'data': 'Bg9CMpleVg3ec/Q1oCrTs46m8pNw+P2mP5Dkuf7

In [22]:
Block("KhlcPujzAs58+e9LT/2XeNueJEbZEvT4efAAtnLD6Uk=")

{'_id': ObjectId('5cc9969e85699a12a228294d'),
 'block': {'header': {'id': {'type': 'SHA256',
    'data': 'KhlcPujzAs58+e9LT/2XeNueJEbZEvT4efAAtnLD6Uk='},
   'timestamp': {'data': 1556715166},
   'previousBlockHash': {'type': 'SHA256',
    'data': 'SWAo24vprvsuLKwkpuKHIlyfxB36QEPh0eUV1CnsGpA='},
   'author': {'signature': {'type': 'SHA256',
     'data': 'MQPzbioZt9p9ie0p43cq3/qMiVGaN54coHIPjqQnZ2nQHF/D5xlpQqn+B1F1cjhW6c1WYleHDDPQ7ChgsPB/9g=='},
    'keyPub': {'type': 'ECP256K1',
     'rawData': 'Awqh/GaK+zXAydO8h4sDf4apFNmW8SFaIqRqXioYDXy8'}},
   'height': 382578}},
 'branch': 'UNVERIFIED',
 'branchPath': {'branchIds': [0], 'blockNumbers': [31]}}